In [1]:
%run supportvectors-common.ipynb



<center><img src="https://d4x5p7s4.rocketcdn.me/wp-content/uploads/2016/03/logo-poster-smaller.png"/> </center>
<div style="color:#aaa;font-size:8pt">
<hr/>
&copy; SupportVectors. All rights reserved. <blockquote>This notebook is the intellectual property of SupportVectors, and part of its training material. 
Only the participants in SupportVectors workshops are allowed to study the notebooks for educational purposes currently, but is prohibited from copying or using it for any other purposes without written permission.

<b> These notebooks are chapters and sections from Asif Qamar's textbook that he is writing on Data Science. So we request you to not circulate the material to others.</b>
 </blockquote>
 <hr/>
</div>



# LangChain Prompts

Good prompts have an overwhelming influence on the quality of the inferences from a large language model. Indeed, over the last few months, there is an increasing realization that the LLMs could potentially start to take over tasks traditionally accomplished through programming -- assuming we can give it well thought out prompts.

<div class="alert-box alert-warning" style="padding-top:30px">
   
<b >Caveat Emptor</b>

> A word of caution is necessary: programming is inherently deterministic, and comprises a clear set of instructions, that yield  -- within certain tolerances -- repeatable results.  </p> On the other hand, large language models are probabilistic machines, and depending on such hyper-parameters as the `temperature`, tend to yield different results on eahc invocation. Here, the value of `temperature` controls the output variations, with `temperature=0` producing close to repeatable results (one reason that this setting is often the default in many situations). On the other hand, higher value, say `temperature=0.9` will tend to produce different results each time.
>
> From our last week's session (our exploration of softmax-temperature), we recall that the temperature is a hyperparameter of the softmax classifier. In other words, when determining the probability of the next word $x_i$ from a vocabulary, the large language models use $$\mathbf{softmax}(x_i/T) = \frac{e^{x_i/T}}{\sum_j e^{x_j/T}}$$ Therefore, the higher the temperature, the more the probability distribution is flattened, and thus a generative model is more likely to sample and pick not necessarily the most likely word, but the next ones in probability. In effect, this leads to a higher variation in the generated text.
<p>
</div>


<hr/>

A `langchain.PromptTemplate` contains two components:

* a template text, with zero or more input-variables embedded in it
* a list of these input variables

Let us now explore a few examples of prompts to develop fluency with them.


### Imports needed for this lab

Let us now import all the components we will need for this lab.

In [2]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate, FewShotPromptTemplate
from langchain.chains import LLMChain
from IPython.display import Markdown, HTML

Consider a prompt that we may want to give a large language model:

<div class="alert-box alert-info" style="padding-top:30px">
    
> Imagine that there is a story about cats. They like to drink milk and prawl in the neighborhood. Write a short story where some cats show these behaviors.
>
> Give the story a catchy, humorous title. Write it in the format:
>        
>        <h3> [Title] </h3>
>        
>        [Body of the story]
    
</div>
As we learned earlier, we can easily do this with:

In [3]:
prompt = """
        Imagine that there is a story about cats. 
        They like to drink milk and prawl in the neighborhood. 
        Write a short story where some cats show these behaviors.
        
        Give the story a catchy, humorous title. Write it in the format:
        
        <h3> [Title] </h3>
        
        [Body of the story]
        """

llm = OpenAI(temperature=0.9)
story = llm(prompt)



In [4]:
display(Markdown('<blockquote>' + story + '</blockquote>'))

<blockquote>
        <h3>Milky Mischief at Midnight</h3>

It was a dark and stormy night when the cats of Mouseville crept out of their homes and into the streets. They wanted nothing more than to get their fill of the sweet, creamy milk they had been dreaming of all day. 

Patches, an old tabby, led the way, tiptoeing between the streetlights, searching for the perfect spot. Suddenly, the light of a nearby porch illuminated a carton of milk on the doorstep.

The cats wasted no time devouring the contents of their find, lapping it up with a furry lip and purring with pleasure. Each lick seemed to bring them more joy than the last.

But the cats weren't done yet. After they had their fill of milk, they scattered throughout the neighborhood, exploring every nook and cranny they could get their little paws on. They peered through windows, under decks, and behind trees. Nothing was beyond their curiosity. 

Just as fast as they had come, the cats of Mouseville vanished into the night, leaving behind only a few droplets of milk and the echoes of their cheery purrs.</blockquote>

#### Generate variations

Since we set the temperature to `temperature=0.9`, we should expect a different story to emerge the next time we call invoke the model with the same input.


In [5]:
story = llm(prompt)
display(Markdown('<blockquote>' + story + '</blockquote>'))

<blockquote>
        <h3>Milk, Mews, and Prowling: Cats in the 'Hood</h3>

In the quiet neighborhood of Sandberry Drive, cats can often be seen lounging around in the sun and exploring the area. They make a stop each night at Mrs. Dabbs' house to get a refreshing drink of milk, and then immediately return to their prowling.

One particular evening, a group of cats led by the oldest of the bunch set out on their nightly adventure. They all leaped and tumbled their way around the streets and backyards, reveling in the delight of their newfound freedom. Every so often, they'd pause to let out a collective meow, as if to announce their presence to the neighborhood.

At Mrs. Dabbs' house, the cats eagerly took their turn to each get a sip of the creamy milk. Fueled by the sustenance, they moved on, sniffing the ground along the way, and chasing their tail as they went.

Before long, the cats reached the edge of a nearby park. Here, they broke off into small groups, taking advantage of every cranny and crevice they could find. The night was filled</blockquote>

**Observations**

* Note how different the two stories about the cats are. This points to the non-determinism of the results.

* We could have set the temperature to `temperature=0`, and seen results that were a bit more similar, yet not identical. We do that below to verify.


In [6]:
zero_llm = OpenAI(temperature=0.0)

# The first version
display (HTML('<h4>Story of cats: Version 1 </h4>'))
story = llm(prompt)
display(Markdown('<blockquote>' + story + '</blockquote>'))

# The second version
display (HTML('<h4> Story of cats: Version 2 </h4>'))
story = llm(prompt)
display(Markdown('<blockquote>' + story + '</blockquote>'))



<blockquote>
        <h3>The Cat's Meow-athon</h3>

It was a warm, summer day in the neighborhood, and the cats were up to their usual antics. All the cats in the area had gathered in the yard of the old Victorian house, where they frolicked and played until their bellies were full and their fur was a mess. 

One particularly daring cat, named Morty, decided to lead the other cats on an adventure, and one by one they gathered around him and followed him to the nearby grocery store. Peeking through the window, they saw the glorious sight of a milk delivery truck, unloading gallons and gallons of milk into the dairy section. 

The cats all cheered in delight, and Morty wasted no time in making his way into the store. As he stumbled upon the milk delivery, a crowd of cats gathered around him, their eyes sparkling with delight. Morty quickly filled the front of his shirt with gallons of milk, before heading out the door and outrunning the store manager. 

After they had their fill of milk, the cats then made their way to the park, where they spent the rest of the day happily prawling around and exploring the area. The cats were exhausted by</blockquote>

<blockquote>
        <h3>The Milk Drinking, Prowling Cats of the Neighborhood</h3>

One day, a group of cats from the neighborhood decided to go out and explore. They were particularly interested in the local grocery store, where they knew milk could be found. 

The cats were careful as they made their way around, keeping their eyes and ears peeled for any signs of danger. When they finally made it to the store, they were delighted to find an array of delicious dairy products. 

One adventurous tabby decided to check out the store's dairy section, while the rest of the cats stayed hidden in the shadows. The tabby snuck around the corner and, before anyone knew it, had lapped up a full carton of milk. After they finished, the cats made their way out of the store as quickly as they had come. 

The cats, now satisfied, spent the rest of the day prowling the neighborhood. They found sun-drenched spots to relax, interesting places to explore, and delicious scraps of food. 

The cats were content with their day of exploration, and had a great story to tell when they got back home. They were forever known as the milk drinking,</blockquote>

## From prompts to templates

Instead of cats, what if we wanted a story about dogs, or about squirrels? A moment of reflection would show that we could have done it with minor variations to our prompt:

**For dogs **

<div class="alert-box alert-info" style="padding-top:30px">   

> Imagine that there is a story about dogs. They like to chew on toys, play fetch and run around the park. Write a short story where some dogs show these behaviors.
>
> Give the story a catchy, humorous title. Write it in the format:
>        
>        <h3> [Title] </h3>
>        
>        [Body of the story]
    
</div>

**For squirrels **

<div class="alert-box alert-info" style="padding-top:30px">   

> Imagine that there is a story about squirrels. They like to eat nuts and fruits, climb up and down trees and run around the glassy meadows. Write a short story where some squirrels show these behaviors.
>
> Give the story a catchy, humorous title. Write it in the format:
>        
>        <h3> [Title] </h3>
>        
>        [Body of the story]
    
</div>

### Capture commonality in templates

Do we see the close similarity between the prompts? Perhaps we would like a lot of stories about animals, and we need to simply give a few descriptive behaviors so the large language models can weave a story around them.

Let us make the above prompts into a template, with two inputs:

* name of the animal
* some characteristic behaviors of the animal


In [7]:
template = """

        Imagine that there is a story about {animal}. 
        They like to {behaviors}. 
        Write a short story where some {animal} show these behaviors.
        
        Give the story a catchy, humorous title. Write it in the format:
        
        <h4> [Title] </h4>
        
        [Body of the story]

        """


Then we can generate our prompt for a story about dogs with:

In [8]:
prompt = PromptTemplate(
            input_variables = ['animal', 'behaviors'],
            template        = template
)

Then, for a story about dogs, we can generate the prompt text as follows:

In [9]:
dogs = 'dogs'
dog_behaviors = 'chew on toys, play fetch and run around the park'

for_dogs = prompt.format(animal = dogs, behaviors = dog_behaviors)
print(for_dogs)



        Imagine that there is a story about dogs. 
        They like to chew on toys, play fetch and run around the park. 
        Write a short story where some dogs show these behaviors.
        
        Give the story a catchy, humorous title. Write it in the format:
        
        <h4> [Title] </h4>
        
        [Body of the story]

        


Likewise, for squirrels, we could get the prompt:

In [10]:
squirrels = 'squirrels'
squirrel_behaviors = """eat nuts and fruits, 
        climb up and down trees and run around the glassy meadows"""

for_squirrels = prompt.format(animal = squirrels, behaviors = squirrel_behaviors)
print (for_squirrels)



        Imagine that there is a story about squirrels. 
        They like to eat nuts and fruits, 
        climb up and down trees and run around the glassy meadows. 
        Write a short story where some squirrels show these behaviors.
        
        Give the story a catchy, humorous title. Write it in the format:
        
        <h4> [Title] </h4>
        
        [Body of the story]

        


#### A simple LangChain 

Let us concatenate the `promptTemplate` and the `llm` into a single chain, so we can invoke it with needing to invoke each component, and feeding its output to the next component.

In [11]:
chain = LLMChain(llm=llm, prompt=prompt)

And now we can invoke it to, say, get a story about dogs:

In [12]:
story = chain.run (animal=dogs, behaviors=dog_behaviors)
display(Markdown('<blockquote>' + story + '</blockquote>'))

<blockquote>
        <h4>A Bark in the Park</h4>

There were four dogs, all eager to get to the park. When they arrived, they couldn't contain their excitement, each looking for something different. 

The first dog, a golden retriever, found a bright red ball and grabbed it in its mouth. It ran around the park, showing off the toy to its friends and eagerly awaiting someone to play fetch.

The second dog, a shih tzu, spotted a bone and scurried over to it. Before anyone had a chance to take it away, it had chewed it into little pieces.

The third dog, a Labrador, just wanted to run around the field. It sprinted back and forth, wagging its tail and happily barking.

Finally, the fourth dog, a beagle, was content simply to wander around and sniff all the different smells.

The four dogs had all enjoyed the park in their own way, and everyone was exhausted by the time they left.</blockquote>

Likewise, a story about squirrels with:

In [13]:
story = chain.run (animal=squirrels, behaviors=squirrel_behaviors)
display(Markdown('<blockquote>' + story + '</blockquote>'))

<blockquote>
        <h4> Nuts and Fruits R Us </h4>

The sun rose over the meadows in the morning, and a small group of squirrels were already bouncing across the grass. They raced around, chasing each other and playing hide and seek. Their excitement was contagious, and soon, even the birds and butterflies joined in the fun.

The squirrels loved to explore and search for food. They scurried up the trees and through the branches, searching for nuts and fruits. As soon as they found something to eat, they quickly devoured it, leaving nothing behind.

When their bellies were full, the squirrels took a break and decided to sunbathe on some of the trees. They rolled around the branches, their tiny little tails wiggling, and their laughter filling the air.

The squirrels spent the day enjoying the fresh air and sunshine, and when night began to fall, they returned back to their homes in the trees. They scampered up the trunks and curled up in their nests, dreaming of the next fun-filled day to come.</blockquote>

**Note**
While a `LLMChain` will take a prompt-template as an argument, it is not necessary to provide a user input. For example, the following prompt template does not take any user input, and still forms a part of the chain, that we can invoke.

In [14]:
simple_prompt = PromptTemplate(
    template="A witty quote on everyday life in contemporary times.",
    input_variables=[] # we cannot omit this argument, even if empty
)
print (simple_prompt.format())

quote = llm(simple_prompt.format())
display(Markdown('<blockquote>' + quote + '</blockquote>'))

A witty quote on everyday life in contemporary times.


<blockquote>

"Life in the 21st century is like a roller coaster ride. You never know when you're going to drop, but you sure have fun when you do!"</blockquote>

#### Chain with a simple prompt

We can run the chain with a simple prompt that needs no user input as follows:

In [15]:
chain = LLMChain(llm=llm, prompt=simple_prompt)
witty_quote = chain.run (input="")
print(witty_quote)



"Life is like a roller coaster. Sometimes you get the thrills, sometimes you get the spills."


## Few shot examples

It is often the case that we want to give the large language models a few exemplars, so that it can infer the relationships. Let us illustrate it with an example.

In [16]:
template = """
Animal: {animal}
Likes to: {likes_to}
"""

examples = [
    {'animal': 'dog', 'likes_to': 'play fetch!'},
    {'animal': 'cat', 'likes_to': 'prawl around the neighborhood'},
    {'animal': 'squirrel', 'likes_to': 'eat nuts and climb trees'}
]


Let us now create an example prompt from this:

In [17]:
example_prompt = PromptTemplate(
    input_variables=['animal', 'likes_to'],
    template = template)


Now, let's create the `FewShotPromptTemplate` with these two.

In [18]:
few_shot_prompt       = FewShotPromptTemplate(
    examples          = examples,
    example_prompt    = example_prompt,
    prefix            = 'Describe the behavior of every animal input',
    suffix            = 'Animal: {input}\nLikes to:',
    input_variables   = ['input'],
    example_separator = '\n\n'
)

We can generate the prompt text from the above:

In [19]:
text = few_shot_prompt.format(input='horse')
print(text)

Describe the behavior of every animal input


Animal: dog
Likes to: play fetch!



Animal: cat
Likes to: prawl around the neighborhood



Animal: squirrel
Likes to: eat nuts and climb trees


Animal: horse
Likes to:


In [20]:
chain = LLMChain(llm=llm, prompt=few_shot_prompt)
chain.run('horse')

' eat hay and run around in fields'

**Summary**

We have learned so far that a `prompt` in `langchain` is the input that goes into a large language model. As we will learn later, there are prompts for other things too, such as chat models, and so forth.

In general, a `prompt` is not necessarily a pre-determined string, but can comprise:

* a template (`PromptTemplate`), with  user input variables embedded
* user inputs for each these variables
* and also some examples


## Partial Prompt Template

Sometimes, we can construct a simpler prompt template by taking one that needs multiple user inputs, but hardwiring some (but not all) of these. Here is an example:

In [21]:
template = """
    Name three historically significant events in {country} around the year {year}
"""

prompt = PromptTemplate (
            input_variables = ['country', 'year'],
            template        = template,
)

print(prompt.format(country='England', year='1800'))


    Name three historically significant events in England around the year 1800



In [22]:
chain = LLMChain(
    llm=llm,
    prompt=prompt
)

history = chain.run(country='England', year='1800')
print(history)


1. The Acts of Union of England and Scotland in 1707 creating the united Kingdom of Great Britain
2. The Industrial Revolution beginning in the late 1700’s
3. The Battle of Trafalgar in October 1805


Let us now suppose that we are particularly interested in the French history.
Then, we can create a simpler, partial prompt out of the above:

In [23]:
partial_prompt = prompt.partial(country='France')
print(partial_prompt.format(year='1789'))


    Name three historically significant events in France around the year 1789



And we can feed it into a language chain as follows:

In [24]:
chain = LLMChain(
            llm     = llm,
            prompt  = partial_prompt
)

events = chain.run(year='1789')
print(events)


1. The Storming of the Bastille (July 14, 1789)
2. Declaration of the Rights of Man and of the Citizen (August 26, 1789)
3. National Assembly Adopts New Constitution (September 3, 1791)


## What we have learned?

In this lab, we learned about:

* what is a `langchain.PromptTemplate`, and what purpose it serves
* many illustrations of constructing the prompt templates
* using prompt templates with few shot examples
* judicious choice of prompt template
